In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import cv2
import glob  
import pandas as pd

path = "/kaggle/input/dl-assignment/label-20210112T150240Z-001/label"
df = pd.DataFrame()

for k in glob.glob(os.path.join(path,'*txt')):
    data1 = pd.read_csv(k, sep=" ", header=None)
    newname = k.replace('.txt', '.jpg')
    newname = newname.replace('/kaggle/input/dl-assignment/label-20210112T150240Z-001/label/', '../input/dl-assignment/image-20210112T150225Z-001/image/')
    data1["file Name"] = newname
    df = df.append(data1,ignore_index=True)

In [3]:
df.columns = ["a", "x1", "y1", "w",'h','File_name','NAN']

In [4]:
df=df.drop(['NAN'], axis=1)

In [5]:
def bbox(df):
    num = random.randint(0, 313)
    path = df.loc[num,'File_name']
    img = cv2.imread(path)
    x1 = df.loc[num,'x1']
    y1 =df.loc[num,'y1']
    x2 = x1+df.loc[num,'w']
    y2 = y1+df.loc[num,'h']
    cv2.rectangle(img, (int(x1),int(y1)), (int(x2),int(y2)), (255,0,0),2)
    plt.imshow(img)
    plt.show()

x1 = targets[0]/299
    y1 = targets[1]/299
    y2 = y1 + targets[3]/299
    x2 = x1+targets[2]/299
    targets= np.array([x1, y1, x2, y2])

In [6]:
df["w"] = df["x1"] + df["w"]
df["h"] = df["y1"] + df["h"]
X_max = df["x1"].max()
y_max = df["y1"].max()
w_max = df["w"].max()
h_max =df["h"].max()
df["x1"] = df["x1"]/df["x1"].max()
df["y1"] = df["y1"]/df["y1"].max()
df["w"] = df["w"]/df["w"].max()
df["h"] = df["h"]/df["h"].max()

df["w"] = df["x1"] + df["w"]
df["h"] = df["y1"] + df["h"]

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(df['File_name'].to_numpy(), df[['x1','y1','w','h']].to_numpy(), test_size=0.2)

X_train, X_val, Y_train, Y_val = train_test_split(X_train,Y_train, test_size=0.2)

In [7]:
X = df['File_name'].to_numpy()
X_test = X[250:282]
X_train = X[:250]
X_val = X[282:]

Y = df[['x1','y1','w','h']].to_numpy()

Y_test = Y[250:282]
Y_train = Y[:250]
Y_val = Y[282:]

In [70]:
df

In [8]:
print(Y_test.shape)
print(Y_train.shape)
print(Y_val.shape)

In [9]:
import tensorflow as tf
AUTOTUNE = tf.data.AUTOTUNE

In [12]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
train_dataset = train_dataset.shuffle(len(X_train))

val_dataset = tf.data.Dataset.from_tensor_slices((X_val, Y_val))

test_dataset = tf.data.Dataset.from_tensor_slices((X_test,Y_test))
test_dataset = test_dataset.shuffle(len(X_test))

In [13]:
def parse_function(images,targets):
    image_string = tf.io.read_file(images)
    image = tf.io.decode_jpeg(image_string,channels=3)
    image = tf.cast(image, tf.float32)
    image = tf.image.convert_image_dtype(image,tf.float32)
    image = tf.image.resize(image,[299, 299])
    image = image / 255
    return image,targets

train_dataset = train_dataset.map(parse_function,num_parallel_calls =AUTOTUNE)

val_dataset = val_dataset.map(parse_function,num_parallel_calls =AUTOTUNE)

test_dataset = test_dataset.map(parse_function,num_parallel_calls =AUTOTUNE)

batch_size = 32

train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)

val_dataset = val_dataset.batch(batch_size)
val_dataset = val_dataset.prefetch(buffer_size=AUTOTUNE)

test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

In [14]:
from keras.models import Sequential, Model
from keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda
from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
#from keras.optimizers import SGD, Adam, RMSprop
from keras.layers.merge import concatenate
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf
import imgaug as ia
from tqdm import tqdm
from imgaug import augmenters as iaa
import numpy as np
import pickle
import os, cv2

In [68]:
early_stop = EarlyStopping(monitor='val_loss', 
                           min_delta=0.001, 
                           patience=3, 
                           mode='min', 
                           verbose=1)

checkpoint = ModelCheckpoint('weights_AAIDL.h5', 
                             monitor='val_loss', 
                             verbose=1, 
                             save_best_only=True, 
                             mode='min', 
                             period=1)

In [15]:
base_model = tf.keras.applications.InceptionResNetV2(include_top=False,weights="imagenet",
                                                                     input_shape=(299, 299, 3),pooling=max)
base_model.trainable = False

model = tf.keras.Sequential([tf.keras.layers.RandomContrast(0.3),
                             base_model,
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dropout(0.3),
                             tf.keras.layers.Dense(16, activation= 'relu'),
                             tf.keras.layers.Dense(4,activation = 'sigmoid')])

In [16]:
import tensorflow_addons as tfa
model.compile(optimizer='adam', loss='mae')
callbacks =[tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=5),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=3, min_lr=0.0001)]
#model.summary()

history = model.fit(train_dataset, verbose=1, validation_data =(val_dataset), shuffle=True,callbacks=callbacks, epochs=100)

In [17]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['training', 'validation'], loc='best')
plt.show()

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
pred = model.predict(test_dataset)
X_test[0]
image1,lable1 = parse_function(X_test[60],Y_test[60])
image1 = tf.reshape(image1, [1,299, 299, 3])
pre = model.predict(image1)
pre

In [20]:
def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0][0], boxB[0])
    yA = max(boxA[0][1], boxB[1])
    xB = min(boxA[0][2], boxB[2])
    yB = min(boxA[0][3], boxB[3])

    # compute the area of intersection rectangle
    interArea = abs(max((xB - xA, 0)) * max((yB - yA), 0))
    if interArea == 0:
        return 0
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = abs((boxA[0][2] - boxA[0][0]) * (boxA[0][3] - boxA[0][1]))
    boxBArea = abs((boxB[2] - boxB[0]) * (boxB[3] - boxB[1]))

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    # return the intersection over union value
    return iou

In [18]:
def bbox(X_test,y_test):
    image1,lable1 = parse_function(X_test,Y_test)
    image1 = tf.reshape(image1, [1,299, 299, 3])
    pre = model.predict(image1)
    
    img = cv2.imread(X_test)
    x1 = y_test[0]*X_max
    y1 =y_test[1]*y_max
    x2 = y_test[2]*w_max
    y2 = y_test[3]*h_max
    x1_pre = pre[0][0]*X_max
    y1_pre = pre[0][1]*y_max
    x2_pre=pre[0][2]*w_max
    y2_pre=pre[0][3]*h_max
    
    
    cv2.rectangle(img, (int(x1),int(y1)), (int(x2),int(y2)), (255,0,0),2)
    cv2.rectangle(img, (int(x1_pre),int(y1_pre)), (int(x2_pre),int(y2_pre)), (0,255,0),2)
    pre = np.array([[x1_pre,y1_pre,x2_pre,y2_pre]])
    y_test = np.array([x1,y1,x2,y2])
    iou = bb_intersection_over_union(pre,y_test)
    cv2.putText(img, "IoU: {:.4f}".format(iou), (10, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    plt.imshow(img)
    plt.show()

In [76]:
def bbox(X_test,y_test):
    image1,lable1 = parse_function(X_test,Y_test)
    image1 = tf.reshape(image1, [1,299, 299, 3])
    pre = model.predict(image1)
    iou = bb_intersection_over_union(pre,y_test)
    img = cv2.imread(X_test)
    x1 = y_test[0]
    y1 =y_test[1]
    x2 = y_test[2]
    y2 = y_test[3]
    x1_pre = pre[0][0]
    y1_pre = pre[0][1]
    x2_pre=pre[0][2]
    y2_pre=pre[0][3]
    cv2.rectangle(img, (int(x1),int(y1)), (int(x2),int(y2)), (255,0,0),2)
    cv2.rectangle(img, (int(x1_pre),int(y1_pre)), (int(x2_pre),int(y2_pre)), (0,255,0),2)
    cv2.putText(img, "IoU: {:.4f}".format(iou), (10, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    plt.imshow(img)
    plt.show()

In [28]:
def bbox(image,lable):
    image1,lable1 = parse_function(image,lable)
    image1 = tf.reshape(image1, [1,299, 299, 3])
    pre = model.predict(image1)
    img = cv2.imread(image)
    x1 = lable[0]
    y1 =lable[1]
    x2 = lable[2]
    y2 = lable[3]
    x2_pre=pre[0][0]+pre[0][2]
    y2_pre=pre[0][1]+pre[0][3]
    print(lable[0].shape)
    iou = bb_intersection_over_union(pre,lable)
    cv2.putText(img, "IoU: {:.4f}".format(iou), (10, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    cv2.rectangle(img, (int(x1),int(y1)), (int(x2),int(y2)), (255,0,0),2)
    cv2.rectangle(img, (int(pre[0][0]),int(pre[0][1])), (int(x2_pre),int(y2_pre)), (0,255,0),2)
    plt.imshow(img)
    plt.show()

In [ ]:
for index in range(len(X_test)):
    img_file = img_list[index]
    label_file = img_file.replace('images', 'labels').replace('jpg', 'txt')
    label_file_open = open(label_file)
    label_file_content = label_file_open.read()
    label_file_coord = np.float_(label_file_content.split())[1:]
    image = cv2.resize(cv2.imread(img_file), (224,224))
    
    image = img_to_array(image)/255.0
    (h,w) = image.shape[:2]
    fig,ax = plt.subplots()
    fig.set_size_inches(10,10)
    image = Image.open(img_file)
    ax.imshow(image)
    
    
    bb_pred = y_pred[index]
    cv2.rectangle(img, (int(bb_pred[0]*w),int(bb_pred[1]*h)), (int(bb_pred[0]*w + bb_pred[2]*w),int(bb_pred[3]*h+bb_pred[1]*h), (255,0,0),2)
    cv2.rectangle(img, (int(label_file_coord[0]*w),int(label_file_coord[1]*h)), (int(label_file_coord[0]*w+label_file_coord[2]*w),int(label_file_coord[1]*h+label_file_coord[3]*h)), (0,255,0),2)
    plt.imshow(img)
    plt.show()

  # cv2.rectangle(img, (int(x1),int(y1)), (int(x2),int(y2)), (255,0,0),2)

  # cv2.rectangle(img, (int(pre[0][0]),int(pre[0][1])), (int(x2_pre),int(y2_pre)), (0,255,0),2)

  # plt.imshow(img)

  # plt.show()

In [ ]:
Y_test[0]

In [22]:
import random

index = random.randint(0, 32)
bbox(X_test[27],Y_test[27])
print(index)
image1,lable1 = parse_function(X_test[2],Y_test[2])
image1 = tf.reshape(image1, [1,299, 299, 3])
pre = model.predict(image1)
print(pre.shape)
iou = bb_intersection_over_union(pre, Y_test[index])
print(iou)

2,6,27,7,25

47,33,2,25,1

In [ ]:
pre = 
bbox(X_test[2],Y_test[2],pre)